In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上进行端到端机器学习：MLOps阶段2：实验：使用Vertex AI超参数调整开始使用XGBoost

<table align="left">
    <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
    
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage2/get_started_vertex_hpt_xgboost.ipynb">
    <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何在Google Cloud上使用Vertex AI进行端到端的MLOps生产操作。本教程涵盖了阶段2：实验：使用Vertex AI超参数调整来开始使用XGBoost。

### 目标

在本教程中，您将学习如何使用 `Vertex AI 超参数调整` 来训练一个 XGBoost 自定义模型。

本教程使用以下谷歌云机器学习服务：

- `Vertex AI 训练`
- `Vertex AI 超参数调整`
- `Vertex AI Vizier`
- `Vertex AI 模型` 资源

执行的步骤包括：

- 使用 Python 包进行训练。
- 在超参数调整时报告准确性。
- 使用 GCSFuse 将模型工件保存到云存储。
- 创建一个 `Vertex AI 模型` 资源。

### 数据集

此教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。该数据集不需要任何特征工程。本教程中的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测三种Iris花的类型：山鸢尾、杂色鸢尾或者维吉尼亚鸢尾。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您预计的使用情况生成成本估算。

安装以下软件包以执行此笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q

重新启动内核

一旦您安装了额外的软件包，您需要重新启动笔记本内核，这样它可以找到这些软件包。

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

##开始之前

###设置您的Google Cloud项目

**以下步骤是必需的，无论您使用的笔记本环境是什么。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得$300的免费信用额度，可用于计算/存储成本。

1. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

1. 如果您是在本地运行此笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行单元格，确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter将以`!`为前缀的行视为shell命令，并将以`$`为前缀的Python变量插入这些命令中。

设置您的项目ID

**如果您不知道您的项目ID**，您可以使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您也可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

不能使用多区域存储桶来训练Vertex AI。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type:"string"}

if REGION == "[your-region]":
    REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试帐户或项目。为了避免用户在创建的资源之间发生名称冲突，您需要为每个实例会话创建一个时间戳，并将时间戳附加到您在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 认证您的 Google Cloud 账户

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经经过身份验证。请跳过此步骤。

**如果您正在使用 Colab**，运行下面的单元格，并按照提示进行身份验证。

**否则**，请按照以下步骤操作：

在 Cloud 控制台中，转到 [创建服务帐号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

1. **单击创建服务帐号**。

2. 在 **服务帐号名称** 栏中输入一个名称，然后单击 **创建**。

3. 在 **将此服务帐号授予项目访问权限** 部分，单击角色下拉列表。在筛选框中输入 "Vertex AI"，然后选择 **Vertex AI 管理员**。在筛选框中输入 "Storage Object Admin"，然后选择 **Storage Object Admin**。

4. 单击创建。包含您的密钥的 JSON 文件将下载到您的本地环境中。

5. 在下面的单元格中将您的服务帐号密钥路径输入为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的笔记本环境如何，以下步骤都是必需的。**

当您为Python初始化Vertex AI SDK时，您需要指定一个云存储临时桶。临时桶是您的数据集和模型资源关联的所有数据在会话之间保留的地方。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中是全局唯一的，包括您的组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

只有你的存储桶不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查云存储桶的内容来验证对其的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip

### 初始化 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 的 Vertex AI SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要使用一个带有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您可以指定：

    （aip.AcceleratorType.NVIDIA_TESLA_K80, 4）

否则，指定（None，None）以使用一个在CPU上运行的容器映像。

了解有关您区域的[硬件加速器支持的更多信息](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在2.3之前的TF版本中，GPU支持将无法在本教程中加载自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由于在serving函数中生成的静态图操作而引起的。如果您在自己的自定义模型上遇到此问题，请使用带有GPU支持的TF 2.3的容器映像。

In [ ]:
import os

if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为训练和预测设置预构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    REGION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 设置变量 `TRAIN_COMPUTE` 来配置您将用于训练的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB的内存。
     - `n1-highmem`: 每个vCPU 6.5GB的内存
     - `n1-highcpu`: 每个vCPU 0.9GB的内存
 - `vCPUs`: 数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意: 以下机器类型不支持用于训练:*

 - `standard`: 2个vCPUs
 - `highcpu`: 2, 4和8个vCPUs

*注意: 您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

## XGBoost 训练简介

一旦您训练了一个 XGBoost 模型，您会希望将其保存在云存储位置，以便随后可以上传到 `Vertex AI Model` 资源。
XGBoost 包不支持将模型保存到云存储位置。相反，您将执行以下步骤将其保存到云存储位置。

1. 将内存中的模型保存到本地文件系统（例如，model.bst）。
2. 使用 gsutil 将本地副本复制到指定的云存储位置。

## Vertex AI 超参数调优服务

以下示例演示了如何使用 Vertex AI 超参数调优服务和 `Vizier` 搜索算法来设置、执行和评估试验。

了解更多关于[超参数调优概述](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview)

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为自定义培训任务组装 Python 包。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到 Docker 镜像的操作环境中的指令。

文件 `trainer/task.py` 是执行自定义培训任务的 Python 脚本。*请注意*，当我们在工作池规范中提到它时，我们将目录斜杠替换为点(`trainer.task`)，并且去掉文件后缀(`.py`)。

#### 包装配

在下面的单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为Python培训包创建任务脚本

接下来，您将为驱动培训包的`task.py`脚本。一些值得注意的步骤包括：

- 命令行参数：
    - `model-dir`：保存训练模型的位置。在使用Vertex AI自定义训练时，位置将在环境变量`AIP_MODEL_DIR`中指定。
    - `dataset_data_url`：要下载的训练数据的位置。
    - `dataset_labels_url`：要下载的训练标签的位置。
    - `boost-rounds`：可调参数。
- 数据预处理（`get_data()`）：
    - 下载数据集并拆分为训练集和测试集。
- 训练（`train_model()`）：
    - 训练模型。
- 评估（`evaluate_model()`）：
    - 评估模型。
    - 如果进行超参数调整，则报告准确度指标。
- 模型工件保存
    - 将模型工件和评估指标保存在由`model-dir`指定的云存储位置。

In [ ]:
%%writefile custom/trainer/task.py
import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb
import hypertune
import argparse
import logging

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--dataset-data-url", dest="dataset_data_url",
                    type=str, help="Download url for the training data.")
parser.add_argument("--dataset-labels-url", dest="dataset_labels_url",
                    type=str, help="Download url for the training data labels.")
parser.add_argument("--boost-rounds", dest="boost_rounds",
                    default=20, type=int, help="Number of boosted rounds")
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def get_data():
    logging.info("Downloading training data and labelsfrom: {}, {}".format(args.dataset_data_url, args.dataset_labels_url))
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_data_url, 'data.csv'], stderr=sys.stdout)
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_labels_url, 'labels.csv'], stderr=sys.stdout)


    # Load data into pandas, then use `.values` to get NumPy arrays
    data = pd.read_csv('data.csv').values
    labels = pd.read_csv('labels.csv').values

    # Convert one-column 2D array into 1D array for use with XGBoost
    labels = labels.reshape((labels.size,))

    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=7)

    # Load data into DMatrix object
    dtrain = xgb.DMatrix(train_data, label=train_labels)
    return dtrain, test_data, test_labels

def train_model(dtrain):
    logging.info("Start training ...")
    # Train XGBoost model
    model = xgb.train({}, dtrain, num_boost_round=args.boost_rounds)
    logging.info("Training completed")
    return model

def evaluate_model(model, test_data, test_labels):
    dtest = xgb.DMatrix(test_data)
    pred = model.predict(dtest)
    predictions = [round(value) for value in pred]
    # evaluate predictions
    accuracy = accuracy_score(test_labels, predictions)
    logging.info(f"Evaluation completed with model accuracy: {accuracy}")

    # report metric for hyperparameter tuning
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='accuracy',
        metric_value=accuracy
    )
    return accuracy


dtrain, test_data, test_labels = get_data()
model = train_model(dtrain)
accuracy = evaluate_model(model, test_data, test_labels)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Export the classifier to a file
gcs_model_path = os.path.join(args.model_dir, 'model.bst')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
model.save_model(gcs_model_path)

logging.info("Saving metrics to {}/metrics.json". format(args.model_dir))
gcs_metrics_path = os.path.join(args.model_dir, 'metrics.json')
with open(gcs_metrics_path, "w") as f:
    f.write(f"{'accuracy: {accuracy}'}")

将培训脚本存储在您的云存储桶中。

接下来，您将训练文件夹打包成压缩 tar 包，然后将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

### 准备您的机器规格

现在为您的自定义超参数调整作业定义机器规格。这告诉 Vertex 需要为超参数调整提供什么类型的机器实例。
  - `machine_type`: 要提供的 GCP 实例的类型 - 例如，n1-standard-8。
  - `accelerator_type`: （如果有的话）硬件加速器的类型。在本教程中，如果您之前设置变量`TRAIN_GPU != None`，则使用GPU；否则将使用CPU。
  - `accelerator_count`: 加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在定义您自定义超参数调整作业的磁盘规格。这将告诉Vertex每台机器实例在超参数调整中要提供的磁盘类型和大小。

- `boot_disk_type`：SSD或Standard。SSD速度更快，Standard价格更便宜。默认为SSD。
- `boot_disk_size_gb`：磁盘大小，以GB为单位。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 准备命令行参数

现在定义自定义训练容器的命令行参数：

- `args`：要传递给设置为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接：您将云存储位置作为命令行参数传递给训练脚本（设置变量`DIRECT = True`），或
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。 在这种情况下，您需要在作业规范中告诉服务模型工件的位置。
  - `--dataset-data-url`：要下载的训练数据的位置。
  - `--dataset-labels-url`：要下载的训练标签的位置。
  - `--boost-rounds`：可调参数。

In [ ]:
MODEL_DIR = "{}/aiplatform-custom-job-{}".format(BUCKET_URI, TIMESTAMP)
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris"


CMDARGS = [
    "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
    "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_URI + "/trainer_iris.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

创建自定义作业

使用`CustomJob`类创建自定义作业，例如用于超参数调整，具有以下参数：

- `display_name`：自定义作业的人类可读的名称。
- `worker_pool_specs`：相应虚拟机实例的规范。
- `base_output_dir`：用于存储模型工件的Cloud Storage位置。

In [ ]:
job = aip.CustomJob(
    display_name="iris_" + TIMESTAMP,
    worker_pool_specs=worker_pool_spec,
    base_output_dir=MODEL_DIR,
)

创建一个超参数调整作业

使用类`HyperparameterTuningJob`来创建一个超参数调整作业，具有以下参数：

- `display_name`：用于自定义作业的人类可读名称。
- `custom_job`：此自定义作业中的工作人员池规范适用于所有试验中创建的CustomJobs。
- `metrics_spec`：要优化的指标。字典键是指标id，由您的训练作业报告，字典值是该指标的优化目标（'minimize'或'maximize'）。
- `parameter_spec`：要优化的参数。字典键是指标id，作为命令行关键字参数传递到您的训练作业中，字典值是指标的参数规范。
- `search_algorithm`：要使用的搜索算法：`grid`，`random`和`None`。如果指定了`None`，则使用`Vizier`服务（贝叶斯）。
- `max_trial_count`：要执行的最大试验次数。

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

hpt_job = aip.HyperparameterTuningJob(
    display_name="iris_" + TIMESTAMP,
    custom_job=job,
    metric_spec={
        "accuracy": "maximize",
    },
    parameter_spec={
        "boost-rounds": hpt.IntegerParameterSpec(min=10, max=100, scale="linear"),
    },
    search_algorithm=None,
    max_trial_count=6,
    parallel_trial_count=1,
)

## Run the hyperparameter tuning job

Use the `run()` method to execute the hyperparameter tuning job.

In [ ]:
hpt_job.run()

### 显示超参数调整作业试验结果

在超参数调整作业完成后，属性 `trials` 将返回每次试验的结果。

In [ ]:
print(hpt_job.trials)

现在看看哪个试验是最好的:

In [ ]:
best = (None, None, None, 0.0)
for trial in hpt_job.trials:
    # Keep track of the best outcome
    if float(trial.final_measurement.metrics[0].value) > best[3]:
        try:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                float(trial.parameters[1].value),
                float(trial.final_measurement.metrics[0].value),
            )
        except:
            best = (
                trial.id,
                float(trial.parameters[0].value),
                None,
                float(trial.final_measurement.metrics[0].value),
            )

print(best)

获取最佳模型

如果使用了用于让服务告诉调整脚本在哪里保存模型工件的方法（`DIRECT = False`），那么最佳模型的模型工件将保存在：

    MODEL_DIR/<best_trial_id>/model

In [ ]:
BEST_MODEL_DIR = MODEL_DIR + "/" + best[0] + "/model"

! gsutil ls {BEST_MODEL_DIR}

删除超参数调整作业

方法'delete()'将删除超参数调整作业。

In [ ]:
hpt_job.delete()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的个别资源：

- 自定义作业（在上一步中删除了自定义训练作业）
- 云存储存储桶

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI